In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
kkkkk



import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

df = pd.read_csv('train.csv')
df.head()

df.drop(columns=['id','CustomerId','Surname'], axis=1, inplace=True)

df

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()


df['Geography']=LE.fit_transform(df['Geography'])
df['Gender']=LE.fit_transform(df['Gender'])



df



df.info()



# df.fillna(df.mean(), inplace=True, numeric_only=True)


x = df.drop('Exited', axis=1)
y = df['Exited']



x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)




model = RandomForestClassifier(n_estimators=250, random_state=42, max_depth=None)
model.fit(x_train, y_train)


y_pred =model.predict(x_test)

#confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.show()

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))




df_test = pd.read_csv('test.csv')
df_test.head()


id = df_test['id']
df_test.drop(columns=['id','CustomerId','Surname'], axis=1, inplace=True)
df_test['Geography']=LE.fit_transform(df_test['Geography'])
df_test['Gender']=LE.fit_transform(df_test['Gender'])
df_test


y_pred_test = model.predict(df_test)



# convert to csv
submission = pd.DataFrame({
    'id': id,
    'Exited': y_pred_test
})
submission.to_csv('submission.csv', index=False)






In [ ]:
mse2

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 📌 Step-2 : Load Data
train = pd.read_csv("/kaggle/input/ai-201-b-mse-2-aiml-b/train.csv")
test = pd.read_csv("/kaggle/input/ai-201-b-mse-2-aiml-b/test.csv")

# 📌 Step-3 : Separate features and target
X = train.drop("NObeyesdad", axis=1)
y = train["NObeyesdad"]

# 📌 Drop id from test
test_features = test.drop("id", axis=1)

# 📌 Step-4 : Handle Missing Values

# Separate numeric and categorical columns
num_cols = X.select_dtypes(exclude='object').columns
cat_cols = X.select_dtypes(include='object').columns

# Imputers
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

# Fit imputers on train, transform both train and test
X[num_cols] = num_imputer.fit_transform(X[num_cols])
test_features[num_cols] = num_imputer.transform(test_features[num_cols])

X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
test_features[cat_cols] = cat_imputer.transform(test_features[cat_cols])

# 📌 Step-5 : Label Encoding Categorical Columns
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test_features[col] = le.transform(test_features[col])
    label_encoders[col] = le

# Encode target
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# 📌 Step-6 : Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)
test_scaled = scaler.transform(test_features)

# 📌 Step-7 : Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 📌 Step-8 : Train Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 📌 Step-9 : Accuracy
pred = model.predict(X_val)
print("Accuracy :", accuracy_score(y_val, pred))


from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix, fbeta_score
import seaborn as sns
import matplotlib.pyplot as plt

# Additional Metrics
precision = precision_score(y_val, pred, average='weighted')
recall = recall_score(y_val, pred, average='weighted')
f1 = f1_score(y_val, pred, average='weighted')

# F2 Score (Gives more importance to recall)
f2 = fbeta_score(y_val, pred, beta=2, average='weighted')

print("\n📊 Additional Evaluation Metrics")
print("---------------------------------")
print("Precision :", precision)
print("Recall    :", recall)
print("F1 Score  :", f1)
print("F2 Score  :", f2)

# Classification Report
print("\n📑 Classification Report")
print(classification_report(y_val, pred, target_names=target_encoder.classes_))

# Confusion Matrix
cm = confusion_matrix(y_val, pred)

plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_encoder.classes_, yticklabels=target_encoder.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.show()

# Feature Importance Graph
importances = model.feature_importances_
feature_names = train.drop("NObeyesdad", axis=1).columns

feat_imp = pd.Series(importances, index=feature_names).sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=feat_imp.values, y=feat_imp.index)
plt.title("Feature Importance (Random Forest)")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()


# 📌 Step-10 : Predict Test
test_pred = model.predict(test_scaled)
test_pred = target_encoder.inverse_transform(test_pred)

# 📌 Step-11 : Submission
submission = pd.DataFrame({
    "id": test["id"],
    "NObeyesdad": test_pred
})
submission.to_csv("submission.csv", index=False)

print("✔ submission.csv saved successfully!")